# 4. 단어 추출 word extraction

## 4-a. 단어 추출을 하는 이유
- 커뮤니티 text는 신조어, 은어가 많다.
- 토큰화를 하기 위해선, 신조어와 은어를 추출해야 한다.

## 4-b. 단어 추출 계획
1. text로만 이루어진 corpus 만들기  
2. soynlp 의 WordExtractor 를 이용하여 단어 추출  
* 기준1 : 특수한 어휘만 추출한다
* 기준2 : 주식이름을 쪼개는 단어는 제외 ex) '페이'는 토스페이를 쪼갠다
4. 단어와 수치 데이터를 csv 파일로 저장  

## 4-c. 단어 추출 진행

### 4-c-1. 파일 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("3_spaced_data.csv", encoding="utf-8")
df.head(3)

,community,gall_id,search_keyword,number,date_created,time_created,author,is_reply,text_length,spaced_text
0,dcinside,snp500,jyp,746844,2023-06-13,12:50:42,ㅇㅇ,0,42.0,jyp가반년만에 배가 올랏네 근디보니는인적자원위주로 하는 사업은먼ㅂㄹ걍제조업이조음
1,dcinside,snp500,jyp,730087,2023-05-16,17:52:40,ㅇㅇ(118.34),0,91.0,야 개보지년들아 jyp 주가 봤냐 미쳣다 지금 개잡주 오르는건 그러려니 하는데 와 ...
2,dcinside,snp500,jyp,730087,2023-05-16,17:52:48,ㅇㅇ(118.34),1,2.0,ㅠㅠ


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053236 entries, 0 to 1053235
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   community       1053236 non-null  object 
 1   gall_id         1053236 non-null  object 
 2   search_keyword  1053236 non-null  object 
 3   number          1053236 non-null  int64  
 4   date_created    1053236 non-null  object 
 5   time_created    1053236 non-null  object 
 6   author          1053236 non-null  object 
 7   is_reply        1053236 non-null  int64  
 8   text_length     1053236 non-null  float64
 9   spaced_text     1053236 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 80.4+ MB


### 4-c-2. corpus를 list 형태로 만든다

In [5]:
corpus = df['spaced_text'].tolist()
len(corpus)

1053236

### 4-c-3. word_extractor 학습시키기

In [6]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(corpus) # list of str or like
words = word_extractor.extract()

training was done. used memory 2.521 Gb use memory 2.992 Gb
all cohesion probabilities was computed. # words = 30097
all branching entropies was computed # words = 1056261
all accessor variety was computed # words = 1056261


In [7]:
words['주식']

Scores(cohesion_forward=0.39677573647604936, cohesion_backward=0.12066574202496533, left_branching_entropy=4.849240032886113, right_branching_entropy=4.652089725485121, left_accessor_variety=606, right_accessor_variety=603, leftside_frequency=50602, rightside_frequency=3393)

In [8]:
len(words)

15035

### 4-c-4. word extraction 결과 출력

In [17]:
# 결과 출력

import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy, word_score)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[6000:7000]:
    print('%s     (%d, %.3f, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy,
            word_score(score)
            )
         )

단어   (빈도수, cohesion, branching entropy, word_score)

내용이     (310, 0.068, 3.483, 2.208)
매수할     (645, 0.103, 3.062, 2.208)
들어가지     (350, 0.203, 2.388, 2.208)
손절은     (169, 0.089, 3.209, 2.208)
프로모션     (108, 0.203, 2.384, 2.207)
없다는     (852, 0.115, 2.952, 2.207)
알서포트     (106, 0.153, 2.671, 2.206)
엑스     (120, 0.106, 3.038, 2.206)
못하게     (419, 0.110, 2.995, 2.205)
국장으로     (100, 0.163, 2.606, 2.204)
니네가     (125, 0.095, 3.146, 2.203)
맥주     (178, 0.137, 2.776, 2.203)
테이퍼링     (129, 0.179, 2.512, 2.202)
양반이     (219, 0.143, 2.736, 2.200)
이마트가서     (111, 0.148, 2.696, 2.200)
비덴트     (254, 0.082, 3.286, 2.200)
울산     (274, 0.125, 2.870, 2.200)
익절함     (118, 0.162, 2.607, 2.198)
따위가     (120, 0.089, 3.210, 2.197)
금양도     (139, 0.080, 3.311, 2.197)
오늘이     (448, 0.065, 3.515, 2.195)
되는걸     (131, 0.068, 3.471, 2.195)
애매한     (265, 0.092, 3.175, 2.195)
이마트를     (219, 0.099, 3.103, 2.194)
세력들     (635, 0.180, 2.497, 2.193)
행복한     (128, 0.181, 2.495, 2.193)
구글은     (117, 0.080, 3.312, 2.19

### 4-c-5. threshold 판단
- 눈으로 직접 살펴보며, 무의미한 결과를 거를 word_score의 threshold를 정한다
- 살펴본 결과. word_score >= 1.620 인 값만 유의미한 데이터로 간주한다
- 결과 dict : scored_words

In [18]:
threshold = 1.620
# threshold 를 기준으로 dictionary 생성
scored_words = {word: word_score(score) for word, score in words.items() if word_score(score) >= threshold}

print(scored_words)

{'잃고': 3.6799728953131545, 'ㅂㅅ': 55.19022162610996, '얼마': 10.035283647353426, '죤나': 16.85802614081082, '병신': 23.662542407964047, '면제': 2.250873876330233, '획득': 8.758480335030798, '존나': 117.97700027594085, '분명': 2.3905151372212465, '디램': 7.527394148309977, '갖고': 5.796388674846454, '승리': 2.6981062916833807, '밑에': 12.764271974649091, '한번': 10.153218644103335, '격차': 6.386682349237054, '사는': 4.394274678925868, '잃은': 2.4940385177161533, '대한': 3.386051387873099, '얻는': 1.6336125327292879, '접고': 3.755689581483339, '훨씬': 43.05477123382923, '읽고': 7.63537237035925, '씨발': 70.54042318997651, '금융': 3.9664764640853925, '독일': 3.902794836952617, '솔까': 6.939843419544632, '매출': 8.158632990430554, '프로': 153.60046508097537, '찾아': 4.02765665575327, '흥행': 4.752254018005701, '담주': 2.5433585020460745, '설명': 2.3616749318421317, '티커': 2.4703877847035125, '휠라': 15.699729506488563, '싸게': 4.853890337711613, '파이': 1.6929338801289897, '플랜': 1.656037873079041, '키트': 1.8380529252305764, '다들': 10.488441953993487, '듣고': 4

In [19]:
len(scored_words)

7001

## 4-d. 단어 정제
1. 영어 포함된 단어 정제
  - 특수한 경우이므로, 프로세스를 나누어 진행한다
2. 조사 포함된 단어 제거
  - "sk텔레콤에" 처럼 조사가 붙은 단어들을 제거한다.
  - 단어 끝에 붙은 조사 처리 : "은", "는", "으로", "이다", "합니다", "였다", "에", "와", "과"
  - list를 뽑아보고, 예외로 처리할 게 있는지 판단한다.
  - "이/가"는 "시가" 같은 멀쩡한 단어를 쪼갤 수 있으니, 진행하지 않거나 주의한다.
    

### 4-d-1. 단어 정제 : 영어가 포함된 단어
- 결과 : scored_words_2

In [20]:
import re

def contains_english(key):
    # 키 값에 영어 알파벳이 하나라도 포함되어 있는지 확인
    return bool(re.search(r'[a-zA-Z]', key))

# 영어 알파벳이 포함된 키들만 추출
english_included_keys = [key for key in scored_words.keys() if contains_english(key)]

print(english_included_keys)
# for key, value in scored_words.items():
#     if contains_english(key):
#         print(f"{key}: {value}")

['pc', 'xx', 'mm', 'ui', 'on', 'kr', 'nc', 'ev', 'or', 'me', 'gs', 'qq', 'ai', 'in', 'it', 'sk', 'ft', 'ww', 'ts', 'kt', 'no', 'us', 'vi', 're', 'ra', 'yg', 'bm', 'op', 'vs', 'sm', 'zz', 'd램', 'dr', 'gm', 'lg', 'gpu', 'krx', 'txt', 'mom', 'baa', 'ddd', 'sdi', 'sns', 'skt', 'ent', 'per', 'hbm', 'byd', 'www', 'cpu', 'suv', 'imf', 'gme', 'oil', 'lg는', 'oem', 'ceo', 'gdp', 'arm', 'jpg', 'ira', 'pvp', 'qld', 'buy', 'amc', 'the', 'nc는', 'enm', 'rsi', 'skc', 'rpg', 'ess', '?id', 'nct', 'cmo', 'mts', 'voo', 'qqq', 'pbr', 'etf', 'jyp', 'oci', 'top', 'eps', 'sk가', 'ssd', 'dcw', 'ipo', 'hmm', 'lg가', 'uam', 'euv', 'amd', 'hts', 'yg는', 'and', 'rea', 'kia', 'sm이', 'roe', 'lgd', 'nft', 'for', 'it주', 'fda', 'ddr', 'app', 'ssg', 'bmw', 'sk는', 'yoy', 'ott', 'gif', 'com', 'ubs', 'bts', 'cpi', 'etf는', 'sk하닉', 'sqqq', 'sk그룹', 'catl', 'feat', 'lg생건', 'lg엔솔', 'tqqq', 'fomc', 'news', 'asml', 'sdi는', '아모레g', 'jyp는', '리니지w', 'sk이노', 'kb증권', 'lg디플', 'pbr이', 'jyp가', 'soxl', 'tsmc', 'fomo', '리니지m', 'etf에', 'thcb',

In [23]:
eng_filter = ['pc', 'xx', 'mm', 'ui', 'on', 'kr', 'nc', 'ev', 'or', 'me', 'gs', 'qq', 
 'in', 'it', 'sk', 'ft', 'ww', 'ts', 'kt', 'no', 'us', 'vi', 're', 
 'ra', 'yg', 'bm', 'op', 'vs', 'sm', 'zz', 'dr', 'gm', 'lg',
 'krx', 'mom', 'baa', 'ddd', 'sns', 'ent', 'hbm', 
 'www', 'cpu', 'suv', 'imf', 'gme', 'oil', 'lg는', 'oem', 'gdp', 
 'ira', 'buy', 'amc', 'the', 'nc는', 'enm', 'rsi', 
 'skc', 'rpg', 'ess', '?id', 'nct', 'cmo', 'mts', 'voo', 'qqq', 'pbr', 'etf', 
 'oci', 'top', 'eps', 'sk가', 'ssd', 'dcw', 'ipo', 'lg가', 'uam', 
 'euv', 'hts', 'yg는', 'and', 'rea', 'kia', 'sm이', 'roe', 'lgd', 'nft', 
 'for', 'fda', 'ddr', 'app', 'ssg', 'bmw', 'sk는', 'yoy', 'ott', 'gif', 
 'com', 'ubs', 'bts', 'cpi', 'etf는', 'catl', 'feat', 
 'news', 'sdi는', 'jyp는', 
 'pbr이', 'jyp가', 'soxl', 'tsmc', 'fomo', 
 'etf에', 'thcb', 'http', 'view', 'soxs', 'etf가',
 'mlcc', 'ceo가', 'per가', 'bbig', 'msci', 'imf때', 'per이', 'per은', 'ftse', 
 'fact', 'sell', 'cciv', 'maga', 'page', 'jyp도', 'jyp에', 
 'dram', 'life', 'board', 'tsmc는', 'https', 'lg엔솔이', 
 'catl이', 'skiet', 'lg화학의', 'lg화학은', 'korea', 'lg화학에', 'lg화학이', 
 'stock', 'jyp엔터', 'kodex', 'naver', 'lg엔솔은', '램시마sc', 
 'lg화학을', 'kospi', 'lg화학도', 'db하이텍', 'lg화학우', 
 'tsmc가', 'tiger', 'lg화학이랑', 
 'lg생활건강', 'mmorpg', 'lg화학에서', 'coming', 'cj제일제당', 'lg화학이나', 
 'ㅋㅋhttps', 'watch?v', 'ㄱㄱhttps', 'sk하이닉스가', 'article', 'sk하이닉스는']

In [24]:
# scored_words 에 적용
def remove_matching_keys(dictionary, key_list):
    return {k: v for k, v in dictionary.items() if k not in key_list}

# 함수 적용
scored_words_2 = remove_matching_keys(scored_words, eng_filter)
len(scored_words_2)

6829

### 4-d-2. 단어 정제 : 조사/접미사가 포함된 단어

In [25]:
scored_words_2

{'잃고': 3.6799728953131545,
 'ㅂㅅ': 55.19022162610996,
 '얼마': 10.035283647353426,
 '죤나': 16.85802614081082,
 '병신': 23.662542407964047,
 '면제': 2.250873876330233,
 '획득': 8.758480335030798,
 '존나': 117.97700027594085,
 '분명': 2.3905151372212465,
 '디램': 7.527394148309977,
 '갖고': 5.796388674846454,
 '승리': 2.6981062916833807,
 '밑에': 12.764271974649091,
 '한번': 10.153218644103335,
 '격차': 6.386682349237054,
 '사는': 4.394274678925868,
 '잃은': 2.4940385177161533,
 '대한': 3.386051387873099,
 '얻는': 1.6336125327292879,
 '접고': 3.755689581483339,
 '훨씬': 43.05477123382923,
 '읽고': 7.63537237035925,
 '씨발': 70.54042318997651,
 '금융': 3.9664764640853925,
 '독일': 3.902794836952617,
 '솔까': 6.939843419544632,
 '매출': 8.158632990430554,
 '프로': 153.60046508097537,
 '찾아': 4.02765665575327,
 '흥행': 4.752254018005701,
 '담주': 2.5433585020460745,
 '설명': 2.3616749318421317,
 '티커': 2.4703877847035125,
 '휠라': 15.699729506488563,
 '싸게': 4.853890337711613,
 '파이': 1.6929338801289897,
 '플랜': 1.656037873079041,
 '키트': 1.83805292523057

In [88]:
len(scored_words_2)

6829

### 4-d-3. 조사/접미사 필터에 추가할 단어 고르기 1
- 하나하나 테스트해본다
- "은/는", "로", "다", "에", "와/과", "이/가", "까지", "부터", "처럼", "도", "을/를", "하", "들"
- 결과 : scored_words_3

In [86]:
# 특정 문자열로 끝나는 단어 리턴
def find_endswith(input_dict, endswith):
    keys_list = list(input_dict.keys())
    result = [item for item in keys_list if item.endswith(endswith)]
    return result
print(find_endswith(scored_words_2, "임"))

['뿐임', '뜻임', '책임', '게임', '주식임', '고점임', '거품임', '안보임', '정도임', '움직임', '수준임', '문제임', '할거임', '될거임', '병신임', '악재임', '놈들임', '기업임', '회사임', '종목임', '생각임', '팩트임', '별로임', '때문임', '얘기임', '닉네임', '느낌임', '호재임', '예정임', '프레임', '살거임', '된거임', '소리임', '치킨게임', '사는거임', '그런거임', '있는거임', '좋아보임', '되는거임', '없어보임', '저평가임', '하는거임', '새끼들임', '서브프라임', '마찬가지임', '말하는거임', '모바일게임']


In [99]:
filter_은는 = [
    '잃은', '많은', '늙은', '깊은', '낮은', '혹은', '번은', '젊은', '것은', '쪽은', '같은', '놓은', '님은', '짤은', '높은', '층은', 
    '답은', '붐은', '낼은', '받은', '옳은', '않은', '곳은', '짧은', '겜은', '남은', '싶은', '놈은', '좋은', '싫은', '작은', '현실은', 
    '산업은', '반등은', '수많은', '당장은', '폭락은', '만원은', '조선은', '놈들은', '애들은', '기업은', '평단은', '게임은', '미장은', 
    '좆같은', '내일은', '쿠팡은', '너같은', '씨젠은', '분들은', '일본은', '삼전은', '쟤들은', '상승은', '수익은', '김정은', '엔솔은', 
    '오늘은', '결론은', '얘들은', '본인은', '내용은', 'ㅂㅅ은', '하닉은', '기관은', '절반은', '배당은', '인텔은', '괜찮은', '라인은', 
    '미국은', '디램은', '한국은', '코인은', '가격은', '회장은', '주식은', '다음은', '시장은', '핵심은', '부분은', '방법은', '비엠은', 
    '생각은', '좃같은', '고점은', '조정은', '애플은', '시총은', '금액은', '중국은', '결국은', '신성은', '공장은', '능력은', '바닥은', 
    '아직은', '손실은', '은행은', '대장은', '실적은', '사람은', '걔들은', '요즘은', '매출은', '금양은', '유럽은', '개같은', '종목은', 
    '판단은', '니들은', '병신은', '한번은', '포홀은', '지금은', '물량은', '삼성은', '외인은', '나같은', '따상은', '익절은', '손절은', 
    '국장은', '비중은', '구글은', '똑같은', '카뱅은', '넥슨은', '일단은', '살아남은', '대부분은', '병신들은', '개미들은', '기업들은', 
    '얘네들은', '셀트같은', '한국장은', '점유율은', '코스닥은', '주식들은', '새끼들은', '수익률은', '주주들은', '가능성은', '사람들은', 
    '에스엠은', '당분간은', '회사들은', '연기금은', '종목들은', '나스닥은', '넷마블은', '조선장은', '서정진은', '대기업은', '직원들은', 
    '기술력은', '매출액은', '부동산은', '아마존은', '신라젠은', '외국인은', '화장품은', '개좆같은', '병신같은', '카카오같은', '검은사막은', 
    '영업이익은', '미국주식은', '현대차같은', '크래프톤은', '한국주식은', '주식시장은', '셀트리온은', '이새끼들은', '마이크론은', '자율주행은', 
    '셀트리온같은', '하이닉스같은', '에코프로같은', '에코프로비엠은', '사는', '얻는', '맞는', '얘는', '하는', '쓰는', '찾는', '잃는', '튀는', 
    '버는', '드는', '먹는', '걔는', '잇는', '타는', '가는', '나는', '또는', '되는', '받는', '않는', '끄는', '보는', '없는', '라는', '있는', 
    '파는', '치는', '쏘는', '쩌는', '듣는', '때리는', '매도는', '이제는', '꼴리는', '낸드는', '회사는', '전에는', '올리는', '욕하는', '흐르는', 
    '키우는', '승자는', '봐서는', '가치는', '매수는', '프로는', '뉴스는', '사라는', '현차는', '주주는', '돈버는', '여기는', '롯데는', '물리는', 
    '누구는', '느끼는', '올해는', '끝나는', '나오는', '해주는', '열리는', '겹치는', '안하는', '로아는', '없다는', '테마는', '원하는', '잡주는', 
    '단타는', '누르는', '걸리는', '땡기는', '목표는', '말고는', '빠지는', '차트는', '우리는', '요새는', '맛있는', '거라는', '당하는', '지수는', 
    '죽이는', '엔씨는', '섹터는', '쳐맞는', '쳐먹는', '싸우는', '말하는', '나가는', '장투는', '일하는', '달리는', '엘화는', '하나는', '어제는', 
    '엘지는', '노조는', '밀리는', '투자는', '얘기는', '거리는', '기아는', '다니는', '걔네는', '있다는', '버티는', '결과는', '이유는', '던지는', 
    '모르는', '빼고는', '된다는', '광기는', '팔리는', '자체는', '엔터는', '즐기는', '깨지는', '고수는', '게이는', '늘리는', '깝치는', '안되는', 
    '했다는', '준다는', '잘하는', '몰리는', '정보는', '새끼는', '굴리는', '좋다는', '경우는', '주가는', '현대는', '다시는', '얘네는', '노리는', 
    '바라는', '줄이는', '개미는', '수요는', '오르는', '쓰이는', '분기는', '업체는', '남자는', '셀트는', '한다는', '뿌리는', '커지는', '돌리는',
    '팩트는', '악재는', '숏치는', '삼바는', '에코는', '뒤지는', '산다는', '소리는', '못하는', '원래는', '말로는', '터지는', '정도는', '간다는', 
    '검사는', '재료는', '바뀌는', '정부는', '만드는', '호재는', '버리는', '두개는', '까지는', '놀리는', '친구는', '보이는', '넘기는', '계좌는', 
    '미래는', '국내는', '규모는', '보다는', '외에는', '망하는', '문제는', '재밌는', '남아있는', '현대차는', '유지하는', '생각나는', '포스코는', 
    '떠오르는', '못만드는', '하겠다는', '망한다는', '기아차는', '폭락하는', '주식하는', '일어나는', '꼴아박는', '예전에는', '테마주는', 
    '떡락하는', '몰빵하는', '인버스는', '삼스디는', '올라가는', '전까지는', '정확히는', '무시하는', '성장하는', '신세계는', '따라가는', 
    '팔아먹는', '먹고사는', '생각하는', '하락하는', '네카오는', '이마트는', '현기차는', '떡상하는', '반도체는', '안나오는', '손해보는', 
    '매도하는', '올라오는', '치료제는', '추천하는', '들어오는', '실제로는', '네이버는', '밀어주는', '매매하는', '들어가는', '관계자는', 
    '컴투스는', '근본없는', '이번주는', '대형주는', '대장주는', '상장하는', '돌아가는', '보여주는', '게임주는', '좆카오는', '누군가는', 
    '날라가는', '투자하는', '꼬라박는', '빅히트는', '안된다는', '빨아먹는', '비교하는', '예상하는', '엔터주는', '사용하는', '아니라는', 
    '개잡주는', '언젠가는', '존버하는', '하이브는', '카카오는', '단타치는', '쓸데없는', '갖고있는', '도요타는', '나온다는', '잘나가는', 
    '예상되는', '아모레는', '준비하는', '수소차는', '운지하는', '급등하는', '시작하는', '착각하는', '돌아오는', '이새끼는', '인정하는', 
    '우량주는', '싫어하는', '살아있는', '알려주는', '좋아하는', '찾아가는', '매수하는', '얘기하는', '발표하는', '박살나는', '테슬라는', 
    '내려가는', '도깨비는', '차전지는', '주남이는', '게임하는', '코스피는', '받아먹는', '리니지는', '납품하는', '오른다는', '바이오는', 
    '존재하는', '판단하는', '빨아주는', '줄어드는', '걱정하는', '앞으로는', '이번에는', '바라보는', '징징대는', '선동하는', '상승하는', 
    '망해가는', '관련주는', '찍어내는', '넘어가는', '수혜주는', '전기차는', '리스크는', '가져가는', '반등하는', '생산하는', '무너지는', 
    '운영하는', '목표가는', '살아남는', '해야하는', '홀딩하는', '손절하는', '의미없는', '나머지는', '기다리는', '공매도는', '인수하는', 
    '배터리는', '기대하는', '메모리는', '들고있는', '떨어지는', '자동차는', '늘어나는', '움직이는', '끌어올리는', '에코프로는', '떨어진다는', 
    '들어가있는', '하이닉스는', '입장에서는', '한국에서는', '에코호로는', '만원까지는', '우상향하는', '삼성전자는', '엔비디아는', '펄어비스는', 
    '엘앤에프는', '바이오주는', '가치투자는', '우리나라는', '카카오뱅크는', '카카오페이는', '장기적으로는', '엔씨소프트는', '단기적으로는', 
    '카카오게임즈는'
]
filter_로다에 = [
    '프로', '바로', '별로', '호로', '브로', '뭘로', '서로', '으로', '걸로', '따로', '상대로', '것으로', '억지로', '제대로', '의외로', 
    '삼프로', '강제로', '종가로', '참고로', '입으로', '그걸로', '선물로', '실수로', '하나로', '실제로', '별개로', '돈으로', '등으로', 
    '그대로', '단체로', '스스로', '진짜로', '엔씨로', '매크로', '필두로', '천으로', '반대로', '댓글로', '맘대로', '확률로', '최초로', 
    '쪽으로', '분할로', '장기로', '셀트로', '담보로', '회사로', '역으로', '투자로', '추가로', '아래로', '식으로', '앞으로', '원으로', 
    '악재로', '논리로', '절대로', '문제로', '규모로', '밑으로', '장투로', '광기로', '속으로', '만으로', '억으로', '엔솔로', '단타로', 
    '말대로', '뉴스로', '매수로', '호재로', '이후로', '함부로', '똑바로', '좆대로', '재미로', '단위로', '몇프로', '단기로', '위주로', 
    '막말로', '공짜로', '달러로', '계좌로', '정도로', '목표로', '테마로', '자회사로', '현금으로', '이상으로', '예상대로', '테슬라로', 
    '배터리로', '시총으로', '이정도로', '배당으로', '갠적으로', '기준으로', '개잡주로', '모바일로', '말그대로', '느낌으로', '가격으로', 
    '코스피로', '현대차로', '차전지로', '에코호로', '한마디로', '삼전으로', '고속도로', '신용으로', '인버스로', '반도체로', '처음으로', 
    '국장으로', '블록딜로', '방식으로', '대량으로', '억원으로', '우량주로', '대상으로', '공매도로', '만원으로', '중심으로', '마음으로', 
    '코로나로', '바탕으로', '세금으로', '상승으로', '생각으로', '자동으로', '희망회로', '진심으로', '목적으로', '에코후로', '그돈으로', 
    '하닉으로', '바이오로', '코인으로', '행복회로', '시간외로', '다음으로', '게임으로', '기점으로', '이런걸로', '미국으로', '기업으로', 
    '다른걸로', '어거지로', '수준으로', '하락으로', '시작으로', '니말대로', '에코프로', '실적으로', '방향으로', '마음대로', '월급으로', 
    '주식으로', '카톡으로', '카카오로', '그정도로', '수익으로', '만원대로', '유니클로', '소액으로', '테마주로', '종목으로', '상장으로', 
    '마인드로', '전기차로', '관련주로', '기반으로', '미장으로', '네이버로', '시장가로', '결과적으로', '기술력으로', '온라인으로', 
    '부정적으로', '장기적으로', '개인적으로', '전반적으로', '실시간으로', '기본적으로', '이런식으로', '자체적으로', '압도적으로', 
    '적립식으로', '지속적으로', '기대감으로', '그런식으로', '펄어비스로', '기술적으로', '안정적으로', '공격적으로', '보수적으로', 
    '상식적으로', '에코프로로', '객관적으로', '대표적으로', '여윳돈으로', '상대적으로', '전체적으로', '하이닉스로', '본격적으로', 
    '긍정적으로', 'ㅋㅋ코프로', '현실적으로', '물적분할로', '인간적으로', '실질적으로', '무지성으로', '단기적으로', '마지막으로', 
    '세계적으로', '하이트진로', '역대급으로', '적극적으로', '마찬가지로', '주기적으로', '카카오페이로', '셀트리온으로', '전세계적으로',
    '잇다', '크다', '쓴다', '판다', '산다', '탔다', '했다', '났다', '쩐다', '됐다', '쏜다', '낫다', '싫다', '떴다', '싸다', '삿다', '싶다', 
    '뜬다', '든다', '난다', '높다', '맞다', '본다', '왔다', '쳤다', '있다', '쉽다', '떳다', '간다', '셋다', '한다', '죄다', '싹다', '낸다', 
    '줬다', '온다', '샀다', '탄다', '둘다', '옳다', '번다', '봤다', '준다', '좋다', '혼다', '갔다', '갖다', '없다', '된다', '같다', '믿는다', 
    '넘는다', '죽는다', '삽니다', '전부다', '흐른다', '갑시다', '넣는다', '끌어다', '담았다', '망했다', '힘들다', '벌었다', '것이다', 
    '드간다', '답없다', '안간다', '않는다', '노가다', '나온다', '중이다', '나보다', '어렵다', '있었다', '보인다', '캐나다', '못한다', 
    '늘었다', '말했다', '다르다', '짓는다', '미쳤다', '못했다', '입니다', '들었다', '좋겠다', '멋지다', '없었다', '잘한다', '버틴다', 
    '올랐다', '좆같다', '슬프다', '빠진다', '박았다', '걸린다', '안산다', '답이다', '망한다', '남았다', '접는다', '말이다', '밝혔다', 
    '아니다', '됩니다', '만간다', '터진다', '멀었다', '때마다', '알겠다', '것보다', '찍는다', '찍었다', '나간다', '그러다', '끝났다', 
    '있겠다', '넘었다', '합니다', '미래다', '잡는다', '먹는다', '거기다', '몰랐다', '최고다', '받는다', '될거다', '오른다', '부럽다', 
    '갑니다', '먹었다', '문제다', '바란다', '뿐이다', '좋겟다', '늦었다', '되겠다', '털었다', '던진다', '회사다', '잘했다', '안된다', 
    '뚫는다', '봅니다', '깨진다', '걸었다', '고맙다', '올린다', '넣었다', '똑같다', '잃었다', '않았다', '합시다', '모른다', '저러다', 
    '편하다', '되었다', '해준다', '팔았다', '옵니다', '돈번다', '말한다', '신이다', '내렸다', '나왔다', '잡았다', '물렸다', '무섭다', 
    '달린다', '심하다', '뒤진다', '받았다', '줄었다', '겁니다', '기회다', '안한다', '빠졌다', '느낀다', '좆됐다', '알았다', '바랍니다', 
    '응원한다', '가능하다', '예상한다', '축하한다', '날라간다', '삼전보다', '몰빵했다', '들고있다', '보입니다', '생각한다', '갈거같다', 
    '증가했다', '왔습니다', '맞습니다', '삼성보다', '종목이다', '잘나간다', '만들었다', '돌아간다', '비슷하다', '든든하다', '미안하다', 
    '안되겠다', '투자한다', '궁금하다', '계획이다', '떡상한다', '예정이다', '아닙니다', '샀습니다', '때문이다', '안됩니다', '다행이다', 
    '횡보하다', '예상보다', '안보인다', '모르겟다', '예상된다', '만원이다', '보고싶다', '상장한다', '들어갔다', '고민된다', '같습니다', 
    '누구보다', '공시했다', '모르겠다', '하닉보다', '지금보다', '들어왔다', '매도했다', '대단하다', '날아간다', '반등한다', '고점이다', 
    '존버하다', '볼때마다', '배아프다', '셀트보다', '움직인다', '물려있다', '승리한다', '매수한다', '불쌍하다', '내려간다', '매수했다', 
    '수준이다', '주식이다', '못하겠다', '불안하다', '위험하다', '만원간다', '있습니다', '말해준다', '생각보다', '부족하다', '무엇보다', 
    '단타치다', '손절했다', '따라간다', '느껴진다', '들어간다', '시작한다', '시작했다', '했습니다', '떨어진다', '없습니다', '싶습니다', 
    '사야겠다', '기다린다', '알려졌다', '해야겠다', '거품이다', '죽고싶다', '기대된다', '추천한다', '드립니다', '폭락한다', '기업이다', 
    '사고싶다', '안타깝다', '한국보다', '기록했다', '전설이다', '좋습니다', '상관없다', '달달하다', '들어온다', '알려준다', '있읍니다', 
    '올라간다', '존버한다', '왔다갔다', '해야한다', '의미없다', '믿습니다', '생각이다', '않습니다', '안늦었다', '것입니다', '인정한다', 
    '저점이다', '필요하다', '확실하다', '시작이다', '느낌이다', '설명했다', '중요하다', '좋아보인다', '풀매수간다', '기원합니다', 
    '감사합니다', '웅장해진다', '카카오보다', '네이버보다', '현대차보다', '예정입니다', '테슬라보다', '가능합니다', '하겠습니다', 
    '여유입니다', '죄송합니다', '응원합니다', '이마트보다', '풀매수했다', '생각합니다', '존경합니다', '불가능하다', '주주입니다', 
    '고맙습니다', '합친거보다', '하이닉스보다', '에코프로보다', '축하드립니다', '부탁드립니다', 
    '밑에', '쪽에', '앞에', '집에', '갤에', '덕에', '층에', '전에', '밖에', '후에', '땜에', '월에', '첨에', '옆에', '위에', '원에', '밤에', '중에', 
    '뒤에', '곳에', '눈에', '년에', '쯤에', '중간에', '만약에', '직전에', '창고에', '년후에', '년안에', '오전에', '때문에', '실적에', '장중에', 
    '엔씨에', '시대에', '년뒤에', '얼마에', '포트에', '애플에', '처음에', '하닉에', '연초에', '증시에', '구글에', '덕분에', '애초에', '둘중에', 
    '서울에', '거기에', '요번에', '악재에', '섹터에', '삼전에', '엔솔에', '새벽에', '산업에', '쿠팡에', '한방에', '주변에', '하루에', '동시에', 
    '연말에', '시간에', '월초에', '년도에', '세상에', '분기에', '계좌에', '여름에', '장초에', '광기에', '살바에', '담주에', '달러에', '검사에', 
    '요즘에', '그전에', '국장에', '월달에', '주가에', '정도에', '가격에', '이번에', '간만에', '금욜에', '생각에', '댓글에', '저녁에', '장전에', 
    '주식에', '엘화에', '년만에', '종목에', '마트에', '고점에', '시초에', '초기에', '은행에', '막판에', '시장에', '초반에', '시외에', '프로에', 
    '월욜에', '아침에', '천원에', '다음에', '셀트에', '분만에', '여기에', '마당에', '기존에', '내년에', '당시에', '최근에', '업체에', '사이에', 
    '바람에', '년전에', '월말에', '과거에', '회사에', '첫날에', '와중에', '오후에', '일만에', '뉴스에', '종가에', '한번에', '게임에', '작년에', 
    '기업에', '예전에', '저번에', '중국에', '나중에', '만원에', '진작에', '주말에', '일전에', '옛날에', '평소에', '경제에', '올해안에', '가기전에', 
    '만원대에', '다음주에', '유튜브에', '오랜만에', '시초가에', '현대차에', '상승장에', '개월만에', '지난주에', '배터리에', '마지막에', '그와중에', 
    '대가리에', '타이밍에', '이번주에', '몇년전에', '순식간에', '공매도에', '얼마전에', '몇달전에', '단기간에', '차전지에', '금요일에', '전기차에', 
    '한달만에', '카카오에', '하루만에', '개잡주에', '바이오에', '가정하에', '이틀만에', '반도체에', '최저점에', '네이버에', '코스피에', '하반기에', 
    '테슬라에', '한달전에', '기대감에', '종토방에', '며칠전에', '넷마블에', '폭락장에', '이와중에', '우량주에', '상반기에', '저번주에', '꼭대기에', 
    '기아차에', '미주갤에', '월요일에', '대한민국에', '검은사막에', '동시호가에', '펄어비스에', '삼성전자에', '에코프로에', '우리나라에', 
    '하이닉스에', '하루아침에', '에코호로에', '셀트리온에', '코로나때문에'
]
filter_와과이가 = [
    '현대차와', '카카오와', '하이닉스와', '셀트리온과',               
    '겜이', '빚이', '짤이', '뜻이', '뿐이', '레이', '야이', '말이', '케이', '페이', '억이', '글이', '쪽이', '짓이', '법이', '라이', '형이', '데이', 
    '돈이', '샤이', '것이', '곳이', '날이', '차이', '각이', '놈이', '님이', '둘이', '등이', '층이', '명이', '힘이', '웨이', '줄이', 
    '씨젠이', '가까이', '플레이', '지들이', '자본이', '한국이', '마음이', '능력이', '라인이', '인생이', '대장이', '특징이', '기술이', 
    '커멜이', '좆같이', '멘탈이', '판단이', '업종이', '반응이', '성장이', '애들이', '형님이', '기업이', '것들이', '세금이', '하락이', '지분이', 
    '공장이', '매출이', '분들이', '여친이', '반등이', '삼성이', '남들이', '자금이', '포홀이', '국민이', '얘들이', '조정이', '산업이', 
    '중국이', '끝없이', '에라이', '지금이', '타격이', '목적이', '걱정이', '직원이', '제약이', '국장이', '버핏이', '댓글이', '실적이', '상장이', 
    '변동이', '운영이', '마진이', '전망이', '가격이', '믿음이', '재앙이', '택진이', '게임이', '모건이', '절반이', '물량이', '확률이', '시장이', 
    '회장이', '상황이', '본인이', '느낌이', '금양이', '거품이', '패턴이', '학식이', '삼전이', '양반이', '내일이', '비엠이', '종목이', 
    '방식이', '경쟁이', '오늘이', '니들이', '배당이', '카뱅이', '내용이', '구글이', '키움이', '효율이', '다같이', '다음이', '기관이', 
    '금액이', '욕심이', '애플이', '손실이', '북한이', '사람이', '방탄이', '엔솔이', '부분이', '세력이', '폭락이', '똑같이', '관심이', 
    '개인이', '일본이', '혁신이', '현금이', '닛케이', '미장이', '주식이', '하닉이', '개같이', '비율이', '제품이', 
    '외인이', '흐름이', '놈들이', '수익이', '비중이', '시총이', '화학이', '넥슨이', '븅신이', '용진이', '수준이', '비용이', 
    '인텔이', '수급이', '틀딱이', '방향이', '신성이', '춘식이', '걔들이', '시간이', '재용이', '약팔이', '주남이', '세상이', '순간이', 
    '의견이', '자산이', '인간이', '임원이', '생각이', '만원이', '영익이', '정권이', '사업이', '쿠팡이', '병신이', '버블이', 
    '환율이', '미국이', '은행이', '희망이', '평단이', '코인이', '전국민이', '모바일이', '포지션이', 
    '직원들이', '화장품이', '민주당이', '유동성이', '바이든이', '애플페이', '서정진이', '최태원이', '조선장이', '테마섹이', 
    '씹스캠이', '외국인이', '경영진이', '기술력이', '회사들이', '하나같이', '순이익이', '상관없이', '에스케이', '스타일이', '제조업이', 
    '정의선이', '여자들이', '주주들이', '대부분이', '거래량이', '기관들이', '병신같이', '용돈벌이', '외인들이', '기대감이', '미친듯이', 
    '디자인이', '엘엔솔이', '점유율이', '한국인이', '병신들이', '연기금이', '유저들이', '몸주남이', '귀신같이', '영향력이', '개인들이', 
    '모멘텀이', '미친놈이', '개미들이', '주식들이', '아마존이', '정용진이', '사이클이', '이유없이', '뜬금없이', '경쟁력이', '업체들이', 
    '기업들이', '종목들이', '타이밍이', '변동성이', '국민들이', '틀딱들이', '문재인이', '나스닥이', '가능성이', '움직임이', '대통령이', 
    '부동산이', '대기업이', '판매량이', '신라젠이', '친구들이', '성장성이', '사람들이', '수익률이', '세력들이', '외궈들이', '새끼들이', 
    '시스템이', '자기들이', '플랫폼이', '부모님이', '코스닥이', '에스엠이', '넷마블이', '생각없이', '이수만이',
    '시가총액이', '대기업들이', '비트코인이', '영업이익이', '국민연금이', '어쩔수없이', '카카오톡이', '붉은사막이', '프로그램이', 
    '셀트리온이', '미국주식이', '크래프톤이', '이새끼들이', '이런애들이', '투자자들이', '검은사막이', '엘지화학이', 
    '주식시장이', '개돼지들이', '자율주행이', '물적분할이', '외국인들이', '진입장벽이', '마이크론이', '아줌마들이', 
    '주남이형이', '영업이익률이', '에코프로비엠이', '아모레퍼시픽이',
    '니가', '먼가', '증가', '퍼가', '제가', '뭐가', '국가', '건가', '뭔가', '누가','추가', '몬가', '드가',  
    '내가', '뇌가', '얘기가', '들어가', '여자가', '갔다가', '기아가', '구조가',  '파이가', 
    '국가가', '나라가', '누군가', '문제가', '새끼가', '삼바가', '게다가', '보다가', '것인가', '머리가', '하다가', '미래가', '숫자가', 
    '상한가', '로아가', '외궈가', '노조가', '자체가', '정부가', '재희가', '달러가', '유저가', '목표가', '정도가', '전체가', '투자가', '단가가', 
    '부모가', '했다가', '악재가', '현차가', '모두가', '걔네가', '개미가', '차이가', '치다가', '지수가', 
    '우리가', '이유가', '테마가', '토스가', '니네가', '올라가', '자리가', '셀트가', '형제가', '짱깨가', '샀다가', '추세가', 
    '차트가', '심리가', '가치가', '남자가', '얘네가', '여기가', '경제가', '친구가', '그런가', '현대가', '업체가', '대표가', '뉴스가', 
    '광기가', '논리가', '시드가',  '격차가', '탔다가', '삿다가', '호재가', '증시가', '어디가', '결과가', '재고가', 
    '신저가', '재료가', '수요가', '엔씨가', '주주가', '액면가', '계좌가', '쓰다가', '자기가', '규모가', '에코가', '두개가', '있다가', 
    '엘화가', '뉴비가', '냄새가', '최고가', '덩치가', '엘지가', '프로가', '회사가', '속도가', '효과가', '주가가', '거래가', '마트가', 
    '위기가', '검사가', '소리가', '섹터가', '슈카가', '따위가', '금리가', '동호가', '적정가', '엄마가', '리니지가', '리스크가', 
    '하는건가', '포스코가', '공매도가', '주린이가', '자동차가', '반도체가', '배터리가', '대다수가', '게임사가', '도깨비가', '코스피가', 
    '카카오가', '수수료가', '컴투스가', '게임주가', '이러다가', '텐센트가', '브랜드가', '올랐다가', '코로나가', '대장주가', '컨텐츠가', 
    '인버스가', '네이버가', '개잡주가', '대가리가', '꼬라지가', '도요타가', '과대평가', '대주주가', '마인드가', '트럼프가', '주남이가', 
    '있는건가', '테슬라가', '이마트가', '경쟁자가', '이미지가', '대형주가', '치료제가', '머스크가', '자회사가', '병신인가', '아저씨가', 
    '빅히트가', '아버지가', '바이오가', '증권사가', '전세계가', '적정주가', '차전지가', '되는건가', '분위기가', '이새끼가', '기아차가', 
    '그런건가', '없는건가', '그러다가', '오르다가', '동시호가', '우량주가', '전기차가', '현대차가', '버티다가', '하이브가', '현기차가', 
    '매수세가', '삼스디가', '목표주가', '거기다가', '삼성전자가', '에코호로가', '게임회사가', '엘앤에프가', '하이닉스가', '우리나라가', 
    '고민하다가', '엔비디아가', '초전도체가', '에코프로가', '펄어비스가', '들어갔다가', '기다렸다가', '카카오페이가', '엔씨소프트가', 
    '카카오뱅크가', '카카오게임즈가'
]

filter_까지부터처럼도을를하들 = [
    '위까지', '월까지', '끝까지', '억까지', '원까지', '말까지', '만까지', '때까지', '일까지', '층까지', '년까지', '퍼까지', '전까지', 
    '아직까지', '현재까지', '내일까지', '내년까지', '여기까지', '만원까지', '지금까지', '얼마까지', '어디까지', '연말까지', '그때까지', 
    '될때까지', '거기까지', '언제까지', '갈때까지', '어제까지', '초반까지', '여태까지', '분기까지', '오늘까지', '나올때까지', '만원대까지',
    '월부터', '일부터', '전부터', '년부터', '낼부터', '때부터', '예전부터', '처음부터', '이제부터', '언제부터', '지금부터', '작년부터', 
    '옛날부터', '그때부터', '벌써부터', '분기부터', '어제부터', '년전부터', '아침부터', '오늘부터', '시작부터', '만원부터', '내년부터', 
    '올해부터', '내일부터', '다음주부터',
    '셀트처럼', '지금처럼', '개잡주처럼', '카카오처럼', '에코프로처럼', '하이닉스처럼', '셀트리온처럼'
    '윈도', '써도', '봐도', '줘도', '년도', '용도', '낼도', '것도', '나도', '정도', '해도', '말고도', '이것도', '회사도', 
    '애플도', '미국도', '매출도', '찍어도', '벌어도', '팔아도', '샀어도', '안해도', '삼전도', '그것도', '프로도', '시총도', '물려도', 
    '전에도', '없어도', '싶어도', '받아도', '누구도', '공장도', '알아도', '주식도', '뒤져도', '보고도', '늦어도', '가격도', '놈들도', '나와도', 
    '조정도', '차트도', '기업도', '금양도', '셀트도', '퍼정도', '있어도', '자체도', '그래도', '억정도', '망해도', '딱봐도', '닌텐도', '중국도', 
    '먹어도', '주가도', '종목도', '천정도', '합쳐도', '한국도', '우리도', '환율도', '니들도', '지금도', '풀매도', '죽어도', '여기도', 
    '투자도', '만원도', '몰라도', '엔씨도', '일본도', '검사도', '저정도', '얘네도', '어제도', '엔솔도', '내일도', '본인도', '이정도', 
    '아직도', '했어도', '올라도', '뭘해도', '못해도', '하닉도', '빠져도', '터져도', '좋아도', '엘화도', '전라도', '삼성도',
    '실적도', '넣어도', '국장도', '월에도', '새끼도', '맞아도', '까지도', '게임도', '애들도', '배당도', '저것도', '잡아도',
    '년정도', '해줘도', '오늘도', '미장도', '어느정도', '테슬라도', '이번에도', '이번년도', '선물매도', '네이버도', '개미들도', 
    '기아차도', '카카오도', '그마저도', '앞으로도', '하는것도', '암만봐도', '기업들도', '넷마블도', '이마트도', '배터리도', '들어가도', 
    '내년에도', '연기금도', '알면서도', '얼마정도', '반도체도', '비교해도', '년전에도', '새끼들도', '너도나도', '중에서도', '안그래도', 
    '차전지도', '하더라도', '현대차도', '누가봐도', '사람들도', '코스닥도', '아니어도', '컴투스도', '후라이도', '공매도도', '아무래도', 
    '바이오도', '이런것도', '아니라도', '프로정도', '만원정도', '하나라도', '거래량도', '전량매도', '생각해도', '떨어져도', '코스피도', 
    '만들어도', '전기차도', '대주매도', '나스닥도', '그렇다쳐도', '에코프로도', '펄어비스도', '조금이라도', '우리나라도', '아니더라도', 
    '마이크론도', '하이닉스도', '아무리봐도', '셀트리온도', '삼성전자도', '지금이라도',
    '것을', '먹을', '돈을', '의견을', '쳐맞을', '종목을', '제품을', '지분을', '물량을', '비중을', '국장을', '성장을', '만원을', '영향을', 
    '기업을', '산업을', '희망을', '흐름을', '공장을', '배당을', '손절을', '기술을', '게임을', '수익을', '주식을', '시장을', '삼전을', '가격을', 
    '실적을', '생각을', '꼬라박을', '점유율을', '기술력을', '살아남을', '플랫폼을', '가능성을', '시스템을', '수익률을', '타이밍을', '셀트리온을',
    '회사를', '자체를', '가치를', '미래를', '소리를', '얼마를', '주가를', '정보를', '매수를', '셀트를', '세계를', '계좌를', '뉴스를', '투자를', 
    '테슬라를', '개소리를', '배터리를', '이마트를', '공매도를', '리스크를', '반도체를', '현대차를', '네이버를', '카카오를', '코스피를', '상한가를', 
    '개잡주를', '바이오를', '서비스를', '기아차를', '전기차를', '삼성전자를', '하이닉스를', '에코프로를', '가치투자를', '펄어비스를',
    '망하', '쩜하', '투자하', '생각하',
    '횽들', '것들', '놈들', '니들', '쟤들', '힘들', '님들', '형들', '걔들', '애들', '얘네들', '개미들', '친구들', '병신들', 
    '게임들', '산애들', '여자들', '임원들', '새기들', '기업들', '주주들', '호구들', '큰손들', '은행들', '잡주들', '행님들', '걔네들', '업체들', 
    '틀딱들', '성님들', '직원들', '외인들', '기관들', '세력들', '본인들', '갤러들', '븅신들', '흑우들', '주식들', '쟤네들', '유저들', '산놈들', 
    '애널들', '인간들', '형님들', '종목들', '회사들', '새키들', '형제들', '새끼들', '사람들', '갤럼들', '노조들', '뉴비들', '외궈들', '짱깨들', 
    '쓰레기들', '광신도들', '전문가들', '미붕이들', '애새끼들', '고수님들', '미친놈들', '투자자들', '아줌마들', '일반인들', '물린놈들', '이새기들', 
    '테슬람들', '물린애들', '부들부들', '셀슬람들', '저새끼들', '증권사들', '자회사들', '대형주들', '하는애들', '관련주들', '개잡주들', '그새끼들', 
    '외국인들', '한국인들', '고인물들', '이새끼들', '우량주들', '대기업들', '린저씨들', '게임사들', '씹새끼들', '개새끼들', '주주님들', '개발자들', 
    '주린이들', '숏충이들', '여기애들', '코붕이들', '개돼지들', '조센징들', '아지매들', '게임주들', '독개미들', '씨발놈들', '게임회사들', 
    '셀트주주들', '씨발새끼들', '스트리머들', '병신새끼들', '호로개미들', '동학개미들', '미친새끼들', '바이오주들', '개미새끼들', '실투갤럼들'
]

filter_etc = ['에코프', '에코프로로', 'ㅋㅋㅋ코프로', '엔비', '에코프로비', '엔비디', '테슬', '초전', '차전', '갑자', '에코프로도', '에코프로의', '에코프로만', '에코프로랑', '에코프로나', '에코도', 'ㅋㅋㅋ캬ㅋㅋㅋ', '공매', '에코랑', '병신들', '병신아', '병신들아', '병신새끼', '개병신', '이새끼들', '저새끼', '이새끼', '셀트리', '곱버', '익절하', '궁금하', '감사합니', '레인보', '생각하']

### 4-d-4. 조사 필터 추가 및 적용 1
- 결과 : scored_words_3
- 단어 6829 -> 4531 개 (2298 개 필터링)

In [77]:
particle_filter = []
particle_filter.extend(filter_은는)
particle_filter.extend(filter_로다에)
particle_filter.extend(filter_와과이가)
particle_filter.extend(filter_까지부터처럼도을를하들)
particle_filter.extend(filter_etc)

# 함수 적용
scored_words_3 = remove_matching_keys(scored_words_2, particle_filter)
len(scored_words_3)

4531

### 4-d-5. 결과 출력 1 : scored_words_3

In [102]:
def print_sorted_dict(my_dict):
    # 사전을 value 기준 내림차순으로 정렬
    sorted_dict = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=True))
    
    # 정렬된 사전의 각 원소를 한 줄에 하나씩 출력
    for key, value in sorted_dict.items():
        print(f"{key}: {value}")

# 함수 호출
print_sorted_dict(scored_words_3)

진짜: 154.71724145318458
맨날: 147.50048913640077
갑자기: 135.93625231057135
솔직히: 135.34116223158497
ㅋㅋ: 133.73484870916687
ㅠㅠ: 130.67295661994297
ㄹㅇ: 127.29026267622237
펄어비스: 126.30905374139093
카카오: 123.27037461070744
근데: 119.5441767936554
존나: 117.97700027594085
셀트리온: 113.86277238737227
ㅜㅜ: 113.84266990799362
ㄷㄷ: 112.42195498657307
컴투스: 105.52481258940043
ㅡㅡ: 104.56052463047064
넷마블: 103.4674070127586
ㅇㅇ: 100.41235094660527
하이닉스: 100.41019013824582
뱅크: 99.65875192939278
현대차: 98.57767479663869
슬슬: 94.98578505380922
캬ㅋㅋ: 93.74878979204406
ㅋㅋㅋ: 92.45471513876186
까지: 92.07417284958026
했는데: 91.19070378101156
ㅉㅉ: 90.39271877683683
너무: 87.57832402278277
반도체: 87.45244894726054
굳이: 87.14507715661776
먼저: 86.16966377110508
sk하이닉스: 85.55622122931918
테슬라: 84.96748131764222
ㅅㅂ: 83.3243935816921
요즘: 83.30088483790995
jyp: 82.56821871371871
얼마나: 81.20223042065999
lg화학: 80.71408870470755
뚜잇: 80.62498975314578
봤는데: 80.04823431859039
결국: 79.45397830032057
혼자: 76.29640449566847
왤케: 75.75125427337025
곱버스: 75.5244

### 4-d-6. 조사/접미사 필터에 추가할 단어 고르기 2
- 조사 하나하나 테스트해본다
- 결과 : scored_words_4

In [174]:
# 특정 문자열로 끝나는 단어 리턴
def find_endswith(input_dict, endswith):
    keys_list = list(input_dict.keys())
    result = [item for item in keys_list if item.endswith(endswith)]
    return result
print(find_endswith(scored_words_3, "듯"))

['할듯', '될듯', '갈듯', '나을듯', '망할듯', '아닌듯', '먹을듯', '있을듯', '빠질듯', '맞는듯', '없을듯', '나올듯', '찍을듯', '힘들듯', '좋을듯', '맞을듯', '오를듯', '좋은듯', '않을듯', '못할듯', '많을듯', '없는듯', '괜찮을듯', '괜찮은듯', '올라갈듯', '가능할듯', '떨어질듯']


In [175]:
filter_special = [
    '돼?', '됨?', '봄?', '응?', '함?', '왜?', '있음?', '안됨?', '읍제?', '같냐?', '어떰?', '좋음?', '뭐임?', '보냐?', '않냐?', '되나?', '뭐냐?', 
    '없제?', 'ㅇㅋ?', '안함?', '머냐?', '망함?', '오름?', '누구?', '되냐?', '얼마?', '뭔데?', '뭐함?', '될까?', '모름?', '뭐지?', 'ㅇㅇ?', '맞나?', 
    '없냐?', 'ㄹㅇ?', '않아?', '가능?', '되지?', '않음?', '맞죠?', '있냐?', 'ㅇㄷ?', '굳이?', '할까?', '알제?', '알지?', '거야?', 'ㄱㅊ?', '했냐?', 
    '왜함?', '하냐?', '살래?', '샀냐?', '낫냐?', '없지?', '올라?', '갈까?', '맞냐?', '정도?', '없나?', '거임?', '않나?', '거냐?', '가냐?', '뭐야?', 
    '없음?', '맞아?', '과연?', '있나?', '아냐?', '몰라?', '실화?', '있어?', '머임?', '믿어?', '좋냐?', '어때?', '맞음?', '뭐다?', '살까?', '왜삼?', 
    '같음?', '아님?', '맞지?', 'ㅇㅈ?', '뭐하냐?', '않았냐?', '얼마임?', '아니냐?', '왜저럼?', '않을까?', '안되냐?', 'ㅅㅌㅊ?', '있었음?', 
    '아닌데?', '될까요?', '아니지?', '생각해?', '카카오?', '가겠지?', '않았음?', '실화냐?', '아닌가?', '현대차?', '나오냐?', '아니노?', '병신임?', 
    '왜오름?', 'ㅁㅌㅊ?', '얼마냐?', '되나요?', '않나요?', '있다고?', '어떠냐?', '떨어짐?', '있는데?', '어케됨?', '가능함?', '그런가?', '있겠냐?', 
    '이유가?', '같은데?', '아니고?', '아니야?', '있을까?', '생각함?', '모르냐?', '뭔가요?', '없겠지?', '회사임?', '오르냐?', '말이됨?', '하겠냐?', 
    '좋을까?', '아닐까?', '산다고?', '오를까?', 'ㅍㅌㅊ?', '가나요?', '안하냐?', '않았나?', '보이냐?', '있나요?', '없는데?', '않을까요?', 
    '아니었냐?', '있을까요?', '하는거임?', '말이되냐?', '아니었음?', '되는거임?', '하이닉스?', '떨어지냐?', '아니겠지?', '오르겠지?', '흑우없제?', 
    '가능할까?', '있는거냐?', '되는거냐?', '생각하냐?', '가능하냐?', '어떤가요?', '셀트리온?', '아닌가요?', '오르는거임?',
    '?!', '간다!', '화이팅!', '가즈아!', '셀트리온!', '감사합니다!'
]    
filter_아에서의만나 = [
    '찾아', '로아', '돌아', '쏟아', '맞아', '잡아', '남아', '팔아', '기아', '받아', '좋아', '시발아', '등신아', '동남아', 'ㅂㅅ아', '씨발아', 
    '똑같아', '없잖아', '얘들아', '거잖아', '되잖아', '븅신아', '했잖아', '애들아', '러시아', '현대기아', '시발련아', '현대위아', '새끼들아', 
    '미친놈아', '시발년아', '븅신들아', '게이들아', '씨발년아', '씨발련아', '아니잖아', '인도네시아', '바이오니아', 
    '씨발련들아', '가즈아아아', '시발련들아', '개새끼들아', '씨발년들아',
    '글에서', '점에서', '갤에서', '선에서', '위에서', '쪽에서', '밑에서', '곳에서', '뒤에서', '옆에서', '중에서', '집에서', '앞에서', '원에서', 
    '층에서', '장에서', '바닥에서', '나라에서', '상황에서', '국장에서', '주갤에서', '조선에서', '게임에서', '주식에서', '사이에서', '한국에서', 
    '만원에서', '방송에서', '주변에서', '회사에서', '하닉에서', '뉴스에서', '기관에서', '세상에서', '업계에서', '삼전에서', '셀트에서', '미국에서', 
    '유럽에서', '중국에서', '국내에서', '측면에서', '분야에서', '과정에서', '일본에서', '가격에서', '코갤에서', '언론에서', '근처에서', '정부에서', 
    '상태에서', '입장에서', '자리에서', '관점에서', '내부에서', '고점에서', '외국에서', '해외에서', '삼성에서', '공장에서', '시점에서', '저점에서', 
    '국가에서', '미장에서', '세계에서', '시장에서', '만원대에서', '실투갤에서', '조선장에서', '증권사에서', '하락장에서', '카카오에서', '현대차에서', 
    '유튜브에서', '최고점에서', '전세계에서', '이마트에서', '인터넷에서', '코스닥에서', '종토방에서', '코스피에서', '셀트리온에서', '하이닉스에서', 
    '펄어비스에서', '우리나라에서', '에코프로에서', '주식시장에서',
    '수준의', '미국의', '한국의', '자신의', '시장의', '일본의', '기업의', '오늘의', '투자의', '미래의', '중국의', '지금의', '최고의', 
    '승리의', '게임의', '규모의', '눈물의', '본인의', '주식의', '회사의', '조선의', '정도의', '개미들의', '반도체의', '테슬라의', '앞으로의', 
    '코스피의', '자본주의', '대부분의', '현대차의', '사람들의', '카카오의', '바이오의', '셀트리온의', '하이닉스의', '펄어비스의', '대한민국의',
    '프로만', '같지만', '조금만', '한번만', '맞지만', '했지만', '삼전만', '좋지만', '셀트만', '두개만', '하지만', '만원만', '골드만', 
    '알지만', '하나만', '않지만', '종목만', '없지만', '까지만', '혼자만', '정도만', '있지만', '개미만', '테슬라만', '카카오만', '있겠지만', 
    '아니지만', '하겠지만', '그렇지만', '않겠지만', '모르지만', '개잡주만', '있었지만', '우량주만', '말하지만', '개미들만', '반도체만', '알겠지만', 
    '없겠지만', '현대차만', '모르겠지만', '하이닉스만', '셀트리온만',
    '되려나', '뭐하나', '벗어나', '않았나', '주식이나', '카카오나', '해야하나', '그렇구나', 
    '말이되나', '있으려나', '렉키로나', '현대차나', '하는구나', '사야하나', '아니었나', '하닉이나', '하이닉스나', '셀트리온이나'
]
filter_거야냐 = [
    '딴거', '쓴거', '큰거', '싼거', '된거', '번거', '높은거', '남은거', '망한거', '먹는거', '넣은거', 
    '싶은거', '다른거', '쓰는거', '이런거', '하는거', '터진거', '받은거', '오른거', '물린거', '타는거', '같은거', '저런거', 
    '받는거', '뜨는거', '빠진거', '있는거', '되는거', '없는거', '올린거', '파는거', '치는거', '붙은거', '버는거', '사는거', '먹을거', '먹은거', 
    '좋은거', '그런거', '가는거', '나오는거', '오르는거', '떨어진거', '돈버는거', '투자한거', '궁금한거', '올리는거', '내리는거', 
    '만드는거', '괜찮은거', '망하는거', '빠지는거', '들어간거', '던지는거', '한다는거', '말하는거', '모르는거', '없다는거', '있다는거', '살만한거', 
    '좆같은거', '손절한거', '비슷한거', '안되는거', '터지는거', '보이는거', '못하는거', '버티는거', '똑같은거', '병신인거', '들어가는거', 
    '매수하는거', '생각하는거', '나락가는거', '움직이는거', '올라가는거', '떨어지는거', '투자하는거', '들어오는거',
    '봐야', '뭐야', '써야', '줘야', '해야', '돼야', '말이야', '잡아야', '있어야', '죽어야', '빠져야', '갚아야', '새끼야', 
    '아니야', '뒤져야', '신이야', '올라야', '해봐야', '망해야', '소리야', '얼마야', '벌어야', '팔아야', '그래야', '넣어야', '있는거야', 
    '하는거야', '병신이야', '지금이야', '이말이야', '뭔소리야', '투자해야', '그래봐야', '이제서야', '개새끼야', '개소리야', '되는거야', 
    '카카오야', '사는거야', '떨어져야', '만들어야', '오르는거야', '병신새끼야',
    '안오르냐', '가는거냐', '어케되냐', '어떡하냐', '병신이냐', '올라가냐', '왜저러냐', '왜이러냐', '떨어지냐', '사는거냐', '되는거냐', 
    '하는거냐', '말이되냐', '개소리냐', '오르는거냐'
]
filter_ㅋㄱㄷㅇㅈㅎㅏㅜㅠㅣ = [
    'ㅌㅋ', 'ㅋㅋㅋ', '?ㅋㅋ', 'ㅌㅋㅋ', '있ㅋㅋ', '어ㅋㅋ', 'ㄴㅋㅋ', '씨ㅋㅋ', '캬ㅋㅋ', '입ㅋㅋ', '시ㅋㅋ', '없ㅋㅋ', '시ㅋㅋㅋ', 'ㅂㅅㅋㅋ', 
    '아니ㅋㅋ', 'ㅋㅋㄱㅋ', 'ㅊㅋㅊㅋ', '캬ㅋㅋㅋ', '어ㅋㅋㅋ', 'ㄹㅇㅋㅋ', 'ㅅㅂㅋㅋ', '병신ㅋㅋ', 'ㅁㅊㅋㅋ', 'ㅋㅋ캬ㅋㅋ', 'ㅅㅂㅋㅋㅋ', 
    'ㄹㅇㅋㅋㅋ', 'ㅇㅈㄹㅋㅋ', '이새끼야ㅋㅋ', 'ㅋㅋㅋ캬ㅋㅋ', '캬ㅋㅋ캬ㅋㅋ',
    'ㄱㄱㄱ', 'ㄷㄷㄷ', 'ㅎㄷㄷㄷ', 'ㅇㅇㅇ', 'ㄹㅇㄹㅇ', 'ㅇㅈㅇㅈ', 'ㅎㅎㅎ',
    '가즈아ㅏㅏ', 'ㅜㅜㅜ', '큐ㅠㅠ', 'ㅠㅠㅠ', 'ㄹㅇㅇㅣ'
]

# filter_etc_ = ['캬ㅋㅋ']

### 4-d-7. 조사 필터 추가 및 적용 2
- 결과 : scored_words_4
- 단어 4531 -> 3953 개 (578 개 필터링)

In [176]:
particle_filter_ = []
particle_filter_.extend(filter_special)
particle_filter_.extend(filter_아에서의만나)
particle_filter_.extend(filter_거야냐)
particle_filter_.extend(filter_ㅋㄱㄷㅇㅈㅎㅏㅜㅠㅣ)
# particle_filter_.extend()


# 함수 적용
scored_words_4 = remove_matching_keys(scored_words_3, particle_filter_)
len(scored_words_4)

3953

### 4-d-8. 결과 확인 Tool

In [173]:
scored_words_4['개소리']

3.086200935454766

In [44]:
# 특정 문자열로 끝나는 단어 리턴
def find_endswith(input_list, endswith):
    result = [item for item in input_list if item.endswith(endswith)]
    return result
def search_dict_keys(input_dict, search_term):
    # 딕셔너리의 키 값을 리스트로 변환
    keys_list = list(input_dict.keys())
    
    # 검색어에 포함되는 키 값을 찾아 새로운 리스트에 추가
    matched_keys = [key for key in keys_list if search_term in key]
    
    # 일치하는 키의 리스트를 반환
    return matched_keys

#### 포함되는 단어 전부 리턴

In [147]:
search_dict_keys(scored_words_4, "ㅇㅈ")

['ㅇㅈㄹ', 'ㅇㅈㅇㅈ', 'ㅇㅈㄹㅋㅋ']

#### 끝나는 단어로 확인

In [130]:
find_endswith(scored_words_4, "ㅋ")

['ㅋㅋ',
 'ㅌㅋ',
 'ㅊㅋ',
 'ㅋㅋㅋ',
 '?ㅋㅋ',
 'ㅌㅋㅋ',
 '있ㅋㅋ',
 '어ㅋㅋ',
 'ㄴㅋㅋ',
 '씨ㅋㅋ',
 '입ㅋㅋ',
 '시ㅋㅋ',
 '없ㅋㅋ',
 '시ㅋㅋㅋ',
 'ㅂㅅㅋㅋ',
 '아니ㅋㅋ',
 'ㅋㅋㄱㅋ',
 'ㅊㅋㅊㅋ',
 '캬ㅋㅋㅋ',
 '어ㅋㅋㅋ',
 'ㄹㅇㅋㅋ',
 'ㅅㅂㅋㅋ',
 '병신ㅋㅋ',
 'ㅁㅊㅋㅋ',
 'ㅋㅋ캬ㅋㅋ',
 'ㅅㅂㅋㅋㅋ',
 'ㄹㅇㅋㅋㅋ',
 'ㅇㅈㄹㅋㅋ',
 '이새끼야ㅋㅋ',
 'ㅋㅋㅋ캬ㅋㅋ',
 '캬ㅋㅋ캬ㅋㅋ']

### 4-d-9. 결과 출력 : scored_words_4

In [177]:
def print_sorted_dict(my_dict):
    # 사전을 value 기준 내림차순으로 정렬
    sorted_dict = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=True))
    
    # 정렬된 사전의 각 원소를 한 줄에 하나씩 출력
    for key, value in sorted_dict.items():
        print(f"{key}: {value}")

# 함수 호출
print_sorted_dict(scored_words_4)

진짜: 154.71724145318458
맨날: 147.50048913640077
갑자기: 135.93625231057135
솔직히: 135.34116223158497
ㅋㅋ: 133.73484870916687
ㅠㅠ: 130.67295661994297
ㄹㅇ: 127.29026267622237
펄어비스: 126.30905374139093
카카오: 123.27037461070744
근데: 119.5441767936554
존나: 117.97700027594085
셀트리온: 113.86277238737227
ㅜㅜ: 113.84266990799362
ㄷㄷ: 112.42195498657307
컴투스: 105.52481258940043
ㅡㅡ: 104.56052463047064
넷마블: 103.4674070127586
ㅇㅇ: 100.41235094660527
하이닉스: 100.41019013824582
뱅크: 99.65875192939278
현대차: 98.57767479663869
슬슬: 94.98578505380922
까지: 92.07417284958026
했는데: 91.19070378101156
ㅉㅉ: 90.39271877683683
너무: 87.57832402278277
반도체: 87.45244894726054
굳이: 87.14507715661776
먼저: 86.16966377110508
sk하이닉스: 85.55622122931918
테슬라: 84.96748131764222
ㅅㅂ: 83.3243935816921
요즘: 83.30088483790995
jyp: 82.56821871371871
얼마나: 81.20223042065999
lg화학: 80.71408870470755
뚜잇: 80.62498975314578
봤는데: 80.04823431859039
결국: 79.45397830032057
혼자: 76.29640449566847
왤케: 75.75125427337025
곱버스: 75.52447670872138
ㅎㅎ: 74.23835340282113
헬스케어: 73.6979

### 4-d-10. 향후 nomalization을 위한 정규화 리스트 만들어두기
- 에코프로 : 에코호로, 에코프로비엠, 에코호로비엠, 에코비엠, 에코후로, 에코형제
- 시발 : ㅅㅂ, ㅆㅂ, 씨발, 씨발년, 씨발련, 시벌, 씨벌, 시불, 씨불
- 병신 : ㅂㅅ, ㅄ, 븅신
- 새끼 : 새끼들, 새기, 세끼
- 현대 : 횬다이
- 존나 : ㅈㄴ, 죤나, 줜나

### 4-d-11. 추가해야할 단어
- ㅇㅈ

## 4-e. 단어 리스트를 파일로 저장

### 4-e-1. 파일로 저장하기

In [178]:
# 파일로 저장
filename = "4_word_list.txt"
words = scored_words_4
with open(filename, "w", encoding="utf-8") as file:
    for word in words:
        file.write(word + "\n")

print("파일 저장 완료.")

파일 저장 완료.


### 4-e-2. 불러와서 확인하기

In [179]:
# 파일로부터 문자열 리스트 불러오기
with open(filename, "r", encoding="utf-8") as file:
    word_list = [line.rstrip('\n') for line in file]

print("불러온 문자열 리스트:")
print(word_list)

불러온 문자열 리스트:
['잃고', 'ㅂㅅ', '얼마', '죤나', '병신', '면제', '획득', '존나', '분명', '디램', '갖고', '승리', '한번', '격차', '대한', '접고', '훨씬', '읽고', '씨발', '금융', '독일', '솔까', '매출', '흥행', '담주', '설명', '티커', '휠라', '싸게', '파이', '플랜', '키트', '다들', '듣고', '고수', '서버', '양산', '믿고', '표현', '토막', '와라', '취업', '농심', '향후', '견제', '줜나', 'ㅌㅌ', '멜론', '좀더', '찍고', '향해', '임상', '복지', '페북', '뜬거', '흑우', '좀만', '힛자', '삽질', '새끼', '쉴드', '겨울', '경제', '큰데', '벨류', '윈도', '좆망', '관심', '싫음', '덩치', '꽤나', '랜덤', '닉값', '거기', '침체', '영익', '서울', '낮음', '축하', '온갖', '싶음', '시장', '숫자', '십년', '횡령', '딱히', '실적', '흐음', 'ㅋㅋ', '쓰고', '쟤네', '없음', '한국', '볼때', '절반', '쉬움', '블소', '버블', '생산', '컨콜', '작전', '틀딱', '패치', '시간', 'ㅈㄴ', '앰창', '빼면', '맥주', '남자', '변화', '임금', '결국', '몰빵', '성장', '역시', '찬성', '세력', '투자', '집단', '벌고', '하닉', '아님', '앨범', 'ㅉㅉ', '털고', '손실', '색히', '몽골', '행님', '냠냠', '줍줍', '음봉', '핵심', '롱숏', '분할', '꼴랑', '목록', '혁신', 'ㅁㅊ', '제일', '퀀텀', '베팅', '완료', '되면', '미친', '꽁돈', '인수', '세금', '될지', '금액', '동안', '습관', '빨리', '혐의', '협상', '시작', '광산', '광고', '필요', '꺼져', '범수', 'ㄹㅇ', '년간', '겨우', '웹

In [182]:
len(word_list)

3953